In [1]:
import psycopg2
from configparser import ConfigParser

def config(filename='prepare_data.ini', section='phonetic'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

def db_connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    # read connection parameters
    params = config()

    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)

    return conn

In [2]:
prepare_english_from_arabic = """
    PREPARE GET_ENGLISH_FROM_ARABIC AS
     SELECT NAME, SUM(COUNT) FROM (
        SELECT SPM_ENAME AS NAME, COUNT(SPM_ENAME) AS COUNT FROM PERSON_MASTER WHERE SPM_ANAME != '' AND SPM_ENAME != '' AND SPM_ANAME = $1 GROUP BY SPM_ENAME
        UNION ALL SELECT SPM_FATHER_ENAME, COUNT(SPM_FATHER_ENAME) FROM PERSON_MASTER WHERE SPM_FATHER_ANAME != '' AND SPM_FATHER_ENAME != '' AND SPM_FATHER_ANAME = $1 GROUP BY SPM_FATHER_ENAME
        UNION ALL SELECT SPM_GRAND_ENAME, COUNT(SPM_GRAND_ENAME) FROM PERSON_MASTER WHERE SPM_GRAND_ANAME != '' AND SPM_GRAND_ENAME != '' AND SPM_GRAND_ANAME = $1 GROUP BY SPM_GRAND_ENAME
        UNION ALL SELECT SPM_POST_GRAND_ENAME, COUNT(SPM_POST_GRAND_ENAME) FROM PERSON_MASTER WHERE SPM_POST_GRAND_ANAME != '' AND SPM_POST_GRAND_ENAME != '' AND SPM_POST_GRAND_ANAME = $1 GROUP BY SPM_POST_GRAND_ENAME
        UNION ALL SELECT SPM_FIFTH_ENAME, COUNT(SPM_FIFTH_ENAME) FROM PERSON_MASTER WHERE SPM_FIFTH_ANAME != '' AND SPM_FIFTH_ENAME != '' AND SPM_FIFTH_ANAME = $1 GROUP BY SPM_FIFTH_ENAME
        UNION ALL SELECT SPM_FAMILY_ENAME, COUNT(SPM_FAMILY_ENAME) FROM PERSON_MASTER WHERE SPM_FAMILY_ANAME != '' AND SPM_FAMILY_ENAME != '' AND SPM_FAMILY_ANAME = $1 GROUP BY SPM_FAMILY_ENAME
        UNION ALL SELECT SPM_MOTHER_ENAME, COUNT(SPM_MOTHER_ENAME) FROM PERSON_MASTER WHERE SPM_MOTHER_ANAME != '' AND SPM_MOTHER_ENAME != '' AND SPM_MOTHER_ANAME = $1 GROUP BY SPM_MOTHER_ENAME
        UNION ALL SELECT SPM_MOTHER_FATHER_ENAME, COUNT(SPM_MOTHER_FATHER_ENAME) FROM PERSON_MASTER WHERE SPM_MOTHER_FATHER_ANAME != '' AND SPM_MOTHER_FATHER_ENAME != '' AND SPM_MOTHER_FATHER_ANAME = $1 GROUP BY SPM_MOTHER_FATHER_ENAME
        UNION ALL SELECT SPM_CLAN_ENAME, COUNT(SPM_CLAN_ENAME) FROM PERSON_MASTER WHERE SPM_CLAN_ANAME != '' AND SPM_CLAN_ENAME != '' AND SPM_CLAN_ANAME = $1 GROUP BY SPM_CLAN_ENAME
    ) AS SUB group BY NAME;
    """

prepare_arabic_from_english = """
    PREPARE GET_ARABIC_FROM_ENGLISH AS
    SELECT NAME, SUM(COUNT) FROM (
        SELECT SPM_ANAME AS NAME, COUNT(SPM_ANAME) AS COUNT FROM PERSON_MASTER WHERE SPM_ANAME != '' AND SPM_ENAME != '' AND SPM_ENAME = $1 GROUP BY SPM_ANAME
        UNION ALL SELECT SPM_FATHER_ANAME, COUNT(SPM_FATHER_ANAME) FROM PERSON_MASTER WHERE SPM_FATHER_ANAME != '' AND SPM_FATHER_ENAME != '' AND SPM_FATHER_ENAME = $1 GROUP BY SPM_FATHER_ANAME
        UNION ALL SELECT SPM_GRAND_ANAME, COUNT(SPM_GRAND_ANAME) FROM PERSON_MASTER WHERE SPM_GRAND_ANAME != '' AND SPM_GRAND_ENAME != '' AND SPM_GRAND_ENAME = $1 GROUP BY SPM_GRAND_ANAME
        UNION ALL SELECT SPM_POST_GRAND_ANAME, COUNT(SPM_POST_GRAND_ANAME) FROM PERSON_MASTER WHERE SPM_POST_GRAND_ANAME != '' AND SPM_POST_GRAND_ENAME != '' AND SPM_POST_GRAND_ENAME = $1 GROUP BY SPM_POST_GRAND_ANAME
        UNION ALL SELECT SPM_FIFTH_ANAME, COUNT(SPM_FIFTH_ANAME) FROM PERSON_MASTER WHERE SPM_FIFTH_ANAME != '' AND SPM_FIFTH_ENAME != '' AND SPM_FIFTH_ENAME = $1 GROUP BY SPM_FIFTH_ANAME
        UNION ALL SELECT SPM_FAMILY_ANAME, COUNT(SPM_FAMILY_ENAME) FROM PERSON_MASTER WHERE SPM_FAMILY_ANAME != '' AND SPM_FAMILY_ENAME != '' AND SPM_FAMILY_ENAME= $1 GROUP BY SPM_FAMILY_ANAME
        UNION ALL SELECT SPM_MOTHER_ANAME, COUNT(SPM_MOTHER_ANAME) FROM PERSON_MASTER WHERE SPM_MOTHER_ANAME != '' AND SPM_MOTHER_ENAME != '' AND SPM_MOTHER_ENAME = $1 GROUP BY SPM_MOTHER_ANAME
        UNION ALL SELECT SPM_MOTHER_FATHER_ANAME, COUNT(SPM_MOTHER_FATHER_ANAME) FROM PERSON_MASTER WHERE SPM_MOTHER_FATHER_ANAME != '' AND SPM_MOTHER_FATHER_ENAME != '' AND SPM_MOTHER_FATHER_ENAME = $1 GROUP BY SPM_MOTHER_FATHER_ANAME
        UNION ALL SELECT SPM_CLAN_ANAME, COUNT(SPM_CLAN_ANAME) FROM PERSON_MASTER WHERE SPM_CLAN_ANAME != '' AND SPM_CLAN_ENAME != '' AND SPM_CLAN_ENAME = $1 GROUP BY SPM_CLAN_ANAME
    ) AS SUB group BY NAME;
    """

conn = db_connect()
try:
    # create a cursor
    cur = conn.cursor()

    # prepare statements
    cur.execute(prepare_english_from_arabic)
    cur.execute(prepare_arabic_from_english)
    
    #execute statement
    print('GET_ENGLISH_FROM_ARABIC')
    cur.execute("execute GET_ENGLISH_FROM_ARABIC('حمزه');")
    
    # display the PostgreSQL database server version
    result = cur.fetchall()
    print(result)
    
    #execute statement
    print('GET_ARABIC_FROM_ENGLISH')
    cur.execute("execute GET_ARABIC_FROM_ENGLISH('HAMZA');")
    
    # display the PostgreSQL database server version
    result = cur.fetchall()
    print(result)
    

    # close the communication with the PostgreSQL
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')

Connecting to the PostgreSQL database...
GET_ENGLISH_FROM_ARABIC
[('GAMZA', Decimal('1')), ('H', Decimal('2')), ('HAMZA', Decimal('27882'))]
GET_ARABIC_FROM_ENGLISH
[('حمزه', Decimal('27882')), ('حمزة', Decimal('8554'))]
Database connection closed.
